This notebook will scrape Wikipedia page to get a table of list of boroughs and neighborhoods in Toronto

install the beautiful soup 4

In [4]:
!pip install beautifulsoup4

     |████████████████████████████████| 102kB 15.2MB/s ta 0:00:01


Import the necessary packages

In [5]:
from bs4 import BeautifulSoup
import pandas as pd
#import requests
import urllib
import urllib.request

use beautiful soup to get the html page

In [16]:
torontourl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
torontopage = urllib.request.urlopen(torontourl)
soup = BeautifulSoup(torontopage, 'html.parser')

find the table and scrape it out from my soup

In [163]:
table = soup.find("table", class_='wikitable sortable')

Get all the rows

In [164]:
rows=table.find_all("tr")

the first row is the labels for the columns

In [86]:
column =[x.text.replace("\n", "") for x in rows[0].find_all("th")]

build a dataframe with the columns. inside here, the boroughs with "Not assigned" are stripped, the neighborhood with "Not assigned" is replaced with the name of the borough. Therefore, it avoids data wrangling at the beginning.

In [162]:
df=pd.DataFrame(columns=column)
for i in range(1, len(rows)):
    tds = rows[i].find_all("td")
    if tds[1].text != 'Not assigned':
        if tds[2].text.strip("\n") == 'Not assigned':
            value = [tds[0].text, tds[1].text, tds[1].text]
        else:
            value = [x.text.strip("\n") for x in tds]
        df = df.append(pd.Series(value, index=column), ignore_index=True)  

Write the dataframe to a csv file before the wrangling

In [116]:
df.to_csv("Toronto.csv", index=True)

group the dataframe accoring to the Postcode. The original dataframe does not have two different boroughs sharing the same postcode.  Therefore, it is save to sort with both postcode and borough

In [160]:
new =df.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list).to_frame()

make the list to string seperated by ","

In [156]:
new["Neighbourhood"] = new["Neighbourhood"].apply(lambda x: ','.join(x))

In [157]:
new.head()

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [158]:
new.to_csv("torontof.csv")

In [159]:
new.shape

(103, 1)